# Introduction

Artem Shlepchenko

as14836

HW_5

Differential fault analysis (DFA) is a cryptanalysis technique that works with most modern cyphers.

Given a plaintext, the DFA attack derived information about the secret key by examining the differences between a related cipher text resulting from a correct encryption and a cipher text of the same plaintext resulting from a faulty encryption.

# DES implementation

DES is a block cipher and encrypts data in blocks of size of 64 bits each, which means 64 bits of plain text goes as the input to DES, which produces 64 bits of ciphertext.

We will start from the DES implementation. Supplementary material was taken from https://en.wikipedia.org/wiki/DES_supplementary_material

The cutInHalves() function cuts input (plaintext) in two halves that thenwill be treated separately.

In [23]:
def cutInHalves(input):
    return (input >> 32) & 0xFFFFFFFF, input & 0xFFFFFFFF

The permutation() function shuffles the bits of a 32-bit half-block.

In [24]:
# Permuts bits of to permute, according to the table
def permutation(toPermute, table , inputSize, verbose=False):
    res = 0
    for i in range(len(table)):
        mask = 1 << (inputSize - table[i])  # The only bit that should be one is the one that will get permuted at this round
        if verbose is True:
            print(bin(mask))
        bitPermuted = bool(toPermute & mask) << (len(table) - i - 1) # then we shift that bit ((bool)toPermute & mask)
                                                                     # at it's post permutation position
        res |= bitPermuted # remember that 0 | 1 = 1, 001 | 100 = 101, etc
    return res

print("Number:", bin(0b1010), hex(0b1010), "table:", [3, 4, 2, 1, 4, 3])
print("permutation:", bin(permutation(0b1010, table=[3, 4, 2, 1, 4, 3], inputSize=4, verbose=True)))

Number: 0b1010 0xa table: [3, 4, 2, 1, 4, 3]
0b10
0b1
0b100
0b1000
0b1
0b10
permutation: 0b100101


The expansion() function is interpreted as for the initial and final permutations. 

In [25]:
E = [
    32, 1,  2,  3,  4,  5,
    4,  5,  6,  7,  8,  9,
    8,  9,  10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

def expansion(inputMessage):
    return permutation(inputMessage, E, 32)

Before the round sub-key is selected, each half of the key schedule state is rotated left by a number of places. This table specifies the number of places rotated.

In [26]:
# left shift a 32 bits integer circularly
def rotate(X):
    poidsFort = 1 if X > pow(2, 27) else 0
    X = X << 1
    X = (X & 0x0FFFFFFF) | poidsFort
    return X

# left shift the two halves of a 32 bits integer circularly
def leftShift(T, verbose=False):
    C = (T & 0xFFFFFFF0000000) >> 28
    D = T & 0x0000000FFFFFFF
    if verbose is True:
        print("before rotation:", hex(C), hex(D))
    C = rotate(C)
    D = rotate(D)
    if verbose is True:
        print("after rotation:", hex(C), hex(D))
    
    C = C << 28
    return (C | D)

a = 0x0000000100000002
print(hex(leftShift(a, True)))
print(hex(rotate(0x10000000)))

before rotation: 0x10 0x2
after rotation: 0x20 0x4
0x200000004
0x1


The PC1 table show which bits from the input key form the left and right sections of the key schedule state. Note that only 56 bits of the 64 bits of the input are selected.

The PC2 table is used to select the 48-bit subkey for each round from the 56-bit key-schedule state.

In [27]:
PC1 = [
    57, 49, 41, 33, 25, 17, 9,
    1,  58, 50, 42, 34, 26, 18,
    10, 2,  59, 51, 43, 35, 27,
    19, 11, 3,  60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
    7,  62, 54, 46, 38, 30, 22,
    14, 6,  61, 53, 45, 37, 29,
    21, 13, 5,  28, 20, 12, 4
]

PC2 = [
    14, 17, 11, 24, 1,  5,
    3,  28, 15, 6,  21, 10,
    23, 19, 12, 4,  26, 8,
    16, 7,  27, 20, 13, 2,
    41, 52, 31, 37, 47, 55,
    30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53,
    46, 42, 50, 36, 29, 32
]

The keySchedule() uses two permutations, PC1 and PC2

PC1 is used once before the loop, in the initialization step. It shrinks the key down from 64 to 56 bit, effectively removing the parity bits. We'll call T that 56 bits version of K

PC2 takes a T as an input and outputs the 48 bits of a Ki. At each step, T is shifted either once or twice (which step gets 2 shifts is hardcoded in the standard too). These shifts ensure every Ki will be different from one another.

Before the round sub-key is selected, each half of the key schedule state is rotated left by a number of places. The array 'nbPermutations' specifies the number of places rotated.

In [28]:
def keySchedule(K):
    subKeys = []
    
    nbPermutations = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]
    T = permutation(K, PC1, 64)

    for perm in nbPermutations:
        T = leftShift(T) if perm == 1 else leftShift(leftShift(T))
        subKeys.append(permutation(T, PC2, 56))
    return subKeys

Each S-box replaces a 6-bit input with a 4-bit output. Given a 6-bit input, the 4-bit output is found by selecting the row using the outer two bits, and the column using the inner four bits.

In [29]:
# There are 8 SBoxes

Sbox = [
    [
        [14, 4,  13, 1,  2,  15, 11, 8,  3,  10, 6,  12, 5,  9,  0,  7],
        [0,  15, 7,  4,  14, 2,  13, 1,  10, 6,  12, 11, 9,  5,  3,  8],
        [4,  1,  14, 8,  13, 6,  2,  11, 15, 12, 9,  7,  3,  10, 5,  0],
        [15, 12, 8,  2,  4,  9,  1,  7,  5,  11, 3,  14, 10, 0,  6,  13]
    ],
    [
        [15, 1,  8,  14, 6,  11, 3,  4,  9,  7,  2,  13, 12, 0,  5,  10],
        [3,  13, 4,  7,  15, 2,  8,  14, 12, 0,  1,  10, 6,  9,  11, 5],
        [0,  14, 7,  11, 10, 4,  13, 1,  5,  8,  12, 6,  9,  3,  2,  15],
        [13, 8,  10, 1,  3,  15, 4,  2,  11, 6,  7,  12, 0,  5,  14, 9]
    ],
    [
        [10, 0,  9,  14, 6,  3,  15, 5,  1,  13, 12, 7,  11, 4,  2,  8],
        [13, 7,  0,  9,  3,  4,  6,  10, 2,  8,  5,  14, 12, 11, 15, 1],
        [13, 6,  4,  9,  8,  15, 3,  0,  11, 1,  2,  12, 5,  10, 14, 7],
        [1,  10, 13, 0,  6,  9,  8,  7,  4,  15, 14, 3,  11, 5,  2,  12]
    ],
    [
        [7,  13, 14, 3,  0,  6,  9,  10, 1,  2,  8,  5,  11, 12, 4,  15],
        [13, 8,  11, 5,  6,  15, 0,  3,  4,  7,  2,  12, 1,  10, 14, 9],
        [10, 6,  9,  0,  12, 11, 7,  13, 15, 1,  3,  14, 5,  2,  8,  4],
        [3,  15, 0,  6,  10, 1,  13, 8,  9,  4,  5,  11, 12, 7,  2,  14]
    ],
    [
        [2,  12, 4,  1,  7,  10, 11, 6,  8,  5,  3,  15, 13, 0,  14, 9],
        [14, 11, 2,  12, 4,  7,  13, 1,  5,  0,  15, 10, 3,  9,  8,  6],
        [4,  2,  1,  11, 10, 13, 7,  8,  15, 9,  12, 5,  6,  3,  0,  14],
        [11, 8,  12, 7,  1,  14, 2,  13, 6,  15, 0,  9,  10, 4,  5,  3]
    ],
    [
        [12, 1,  10, 15, 9,  2,  6,  8,  0,  13, 3,  4,  14, 7,  5,  11],
        [10, 15, 4,  2,  7,  12, 9,  5,  6,  1,  13, 14, 0,  11, 3,  8],
        [9,  14, 15, 5,  2,  8,  12, 3,  7,  0,  4,  10, 1,  13, 11, 6],
        [4,  3,  2,  12, 9,  5,  15, 10, 11, 14, 1,  7,  6,  0,  8,  13]
    ],
    [
        [4,  11, 2,  14, 15, 0,  8,  13, 3,  12, 9,  7,  5,  10, 6,  1],
        [13, 0,  11, 7,  4,  9,  1,  10, 14, 3,  5,  12, 2,  15, 8,  6],
        [1,  4,  11, 13, 12, 3,  7,  14, 10, 15, 6,  8,  0,  5,  9,  2],
        [6,  11, 13, 8,  1,  4,  10, 7,  9,  5,  0,  15, 14, 2,  3,  12]
    ],
    [
        [13, 2,  8,  4,  6,  15, 11, 1,  10, 9,  3,  14, 5,  0,  12, 7],
        [1,  15, 13, 8,  10, 3,  7,  4,  12, 5,  6,  11, 0,  14, 9,  2],
        [7,  11, 4,  1,  9,  12, 14, 2,  0,  6,  10, 13, 15, 3,  5,  8],
        [2,  1,  14, 7,  4,  10, 8,  13, 15, 12, 9,  0,  3,  5,  6,  11]
    ]
]

F() is a function parameterized with a 48-bit key and operating on a 32-bit block

In [30]:
P = [
    16, 7,  20, 21,
    29, 12, 28, 17,
    1,  15, 23, 26,
    5,  18, 31, 10,
    2,  8,  24, 14,
    32, 27, 3,  9,
    19, 13, 30, 6,
    22, 11, 4,  25
]

def F(R, subkey):
    # after expansion, R is 48 bits long
    T = expansion(R) ^ subkey
    
    res = 0

    for i in range(1, 9):
        # 48 bits / 8 = 8 block of 6 bits to be handle separately 
        block = T >> (48 - i*6)

        msbDoubled = (block & 0b100000) >> 4
        lsb = block & 1

        row = msbDoubled + lsb
        column = (block & 0b011110) >> 1

        # Then every block goes throught one SBox => 4 bits outputed
        # for each Sbox, append it's output to the result
        res = res | (Sbox[i - 1][row][column] << (32 - i*4))
    return permutation(res, P, 32)

The IP table specifies the input permutation on a 64-bit block. The meaning is as follows: the first bit of the output is taken from the 58th bit of the input; the second bit from the 50th bit, and so on, with the last bit of the output taken from the 7th bit of the input.

The IPinv (the final permutation) is the inverse of the initial permutation.

In [31]:
IP = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9,  1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]
    
IPinv = [
    40, 8,  48, 16, 56, 24, 64, 32,
    39, 7,  47, 15, 55, 23, 63, 31,
    38, 6,  46, 14, 54, 22, 62, 30,
    37, 5,  45, 13, 53, 21, 61, 29,
    36, 4,  44, 12, 52, 20, 60, 28,
    35, 3,  43, 11, 51, 19, 59, 27,
    34, 2,  42, 10, 50, 18, 58, 26,
    33, 1,  41, 9,  49, 17, 57, 25
]

Finally, we can inplement DES function 

In [32]:
def DES(clear, K):
    # Derivating K into 16 subkeys
    subKeys = keySchedule(K)
    # Inital Permutation, IP
    clearIP = permutation(clear, IP, 64)
    L, R = cutInHalves(clearIP)

    # 1 Feistel round/subkey
    for i in subKeys:
        LiPlus1 = R
        RiPlus1 = L ^ F(R, i)

        L = LiPlus1
        R = RiPlus1
        #print(hex(L)) # uncomment this line
        #print(hex(R)) # and this line
    # swap R16 and L16
    swapped = (R << 32) | L
    # apply IP^-1
    return permutation(swapped, IPinv, 64)

and run...

In [33]:
clear = 0x1274775212747752    
key   = 0x2577472125774721
print("Cleartext = " + hex(clear) + " | Key = " + hex(key) + " | Cypher : " + hex(DES(clear, key)))

Cleartext = 0x1274775212747752 | Key = 0x2577472125774721 | Cypher : 0xbee7152a9320a971


To verify output of this implementation we can use online DES calculator http://www.emvlab.org/descalc/

# Differential fault analysis (DFA)

Initially, we need to load some plaintext, ciphertext and 32 faulty ciphertexts.
The faulty ciphertextt are the result of 32 executions of DES where a bit has been changed on the R15 block and then has been fed to the last call to the F function (which used the `K16` subkey).

In [34]:
plaintext  = 0x54bd2a11e5938c37 
ciphertext = 0xf2ea8be913629cbd
print("Plaintext: ", hex(plaintext))
print("Ciphertext:", hex(ciphertext))

faultedCyphertexts = [
    0xf0fb8be913639cb9, 0xf2e88bed13639cbd, 0xf2ea89a913639cbd, 0xf3fa8fef03639cbd, 
    0xf3fa8bed01629cbd, 0xf3aa8be903609cbd, 0xf2aa8fe903629ebd, 0xf3aa8fe803729cbf, 
    0xfbea8be803669cbd, 0xf2e28be853729cbd, 0xf2ea83e853669cbd, 0xf2ea9be053229cbc, 
    0xf2ea9be85b329cbd, 0xb2ea9be9132a9cbd, 0xb2ea8be9136294bc, 0xb2ea9be917629cf4, 
    0xd2ea8be913228cfc, 0xf2ca8be917628cbd, 0xf2eaabe917629cfd, 0xf6eacbc913629dfd, 
    0xe6eacbe937629dfd, 0xf6ea8ae913429cbd, 0xe2eacae91362bcbd, 0xf2eacaf91262dc9d, 
    0x76eacbf9126298bd, 0xf26a8bf912629cbd, 0xf2ea0bf91362d8bd, 0xf2ef8b691262dcbd,
    0xf2ef8be99262d8b9, 0xf2eb8be913e29cb9, 0xf2ee8be913621cb9, 0xf2ef8ba913639c2d
]
print("Faulted cyphertexts:", [hex(i) for i in faultedCyphertexts])

Plaintext:  0x54bd2a11e5938c37
Ciphertext: 0xf2ea8be913629cbd
Faulted cyphertexts: ['0xf0fb8be913639cb9', '0xf2e88bed13639cbd', '0xf2ea89a913639cbd', '0xf3fa8fef03639cbd', '0xf3fa8bed01629cbd', '0xf3aa8be903609cbd', '0xf2aa8fe903629ebd', '0xf3aa8fe803729cbf', '0xfbea8be803669cbd', '0xf2e28be853729cbd', '0xf2ea83e853669cbd', '0xf2ea9be053229cbc', '0xf2ea9be85b329cbd', '0xb2ea9be9132a9cbd', '0xb2ea8be9136294bc', '0xb2ea9be917629cf4', '0xd2ea8be913228cfc', '0xf2ca8be917628cbd', '0xf2eaabe917629cfd', '0xf6eacbc913629dfd', '0xe6eacbe937629dfd', '0xf6ea8ae913429cbd', '0xe2eacae91362bcbd', '0xf2eacaf91262dc9d', '0x76eacbf9126298bd', '0xf26a8bf912629cbd', '0xf2ea0bf91362d8bd', '0xf2ef8b691262dcbd', '0xf2ef8be99262d8b9', '0xf2eb8be913e29cb9', '0xf2ee8be913621cb9', '0xf2ef8ba913639c2d']


The selectFaults() function will help to denote which fault went into which SBox:

In [35]:
def selectFaults():
    aAtq = {'sbox{}'.format(i): [] for i in range(8)}

    for index, i in enumerate(faultedCyphertexts):
        xor = ciphertext ^ i
        xor = permutation(xor, IP, 64)
        
        L16, R16 = cutInHalves(xor)
        exp = bin(expansion(R16))[2:].zfill(48)
        expListe = [exp[i:i+6] for i in range(0, 48, 6)]
        
        for idx, bloc in enumerate(expListe):
            if bloc != '000000':
                aAtq['sbox{}'.format(idx)].append(index)
    return aAtq

print(selectFaults())

{'sbox0': [0, 27, 28, 29, 30, 31], 'sbox1': [23, 24, 25, 26, 27, 28], 'sbox2': [19, 20, 21, 22, 23, 24], 'sbox3': [15, 16, 17, 18, 19, 20], 'sbox4': [11, 12, 13, 14, 15, 16], 'sbox5': [7, 8, 9, 10, 11, 12], 'sbox6': [3, 4, 5, 6, 7, 8], 'sbox7': [0, 1, 2, 3, 4, 31]}


Helper functions

In [36]:
# Find the common point of a list of lists
def common(list):
    result = set(list[0])
    for l in list[1:]:
        result.intersection_update(l)
    return result.pop()

# Isolates the input of the sbox by generating a 6 bits mask
def mask48(sbox):
    mask = "111111"
    zerosGauche = sbox * 6 * '0'
    mask = zerosGauche + mask
    while(len(mask) < 48):
        mask += '0'
    return int(mask, 2)

# Isolates the output of the sbox by generating a 4 bits mask
def mask32(sbox):
    mask = "1111"
    return int(mask + "0" * (28 - sbox * 4), 2)

def calcRowColumnSbox(sbox, expanded, valueToTest):
    # only keep the 6 bits that go throught the sbox
    tmp = expanded & mask48(sbox)
    # removing the zeros on the right
    tmp >>= (7 - sbox) * 6
    tmp ^= valueToTest  # valueToTest is the potential value of the current block of K16 that we're testing (0 to 63)
    
    # same way to calculate the line/column of the input as in DES
    msbDouble = (tmp & 0b100000) >> 4
    lsb = tmp & 1
    row = msbDouble + lsb 
    column = (tmp & 0b011110) >> 1
    return row, column

Pinv = [
    9,  17, 23, 31,
    13, 28, 2,  18,
    24, 16, 30, 6,
    26, 20, 10, 1,
    8,  14, 25, 3,
    4,  29, 11, 19,
    32, 12, 22, 7,
    5,  27, 15, 21
]

def solutionIsValid(sbox, P_1_L16_xor_L16f, line, column, linef, columnf):
    ver = P_1_L16_xor_L16f & mask32(sbox) # keep the (4 bits) output of the current S box
    ver >>= (7 - sbox) * 4            # remove trailing zeros
    xor = Sbox[sbox][line][column] ^ Sbox[sbox][linef][columnf] 
    return ver == xor                 # if S(E(R15) ^ x) ^ S(E(R15*) ^ x) = P-1(L16 ^ L16*) then we found a potential solution

Now we can find key k16

In [37]:
import pprint

def findK16(ciphertext, faultedCyphertexts, verbose=False):
    pp = pprint.PrettyPrinter(indent=4)
    K16 = 0x000000000000
    sol = {"sbox{}".format(i): [] for i in range(8)}

    toAttack = selectFaults()
    
    if verbose is True:
        print("Indexes of the SBoxes to be attacked:")
        pp.pprint(toAttack)
        print()
    
    # Replacing the indexes with the values
    for sbox, lists in toAttack.items():
        listFaultedCyphertexts = []
        for i in lists:
            listFaultedCyphertexts.append(faultedCyphertexts[i])
        toAttack[sbox] = listFaultedCyphertexts
    
    L16, R15 = cutInHalves(permutation(ciphertext, IP, 64))
    
    # Attacking the 8 sboxes with the equation we found
    for s in range(8):
        # Each SBox is attacked by 6 faulted cyphers
        for f in range(6): 
            L16f, R15f = cutInHalves(permutation(toAttack['sbox{}'.format(s)][f], IP, 64))
    
            # calculating the terms of the equation
            P_1_L16_xor_L16f = permutation(L16 ^ L16f, Pinv, 32)
            E_R15 = expansion(R15) 
            E_R15f = expansion(R15f)
            
            # currSols stocks a list of possible solutions for the current sbox and faulted cypher
            currSols = []
            # trying every values for the current portion of K16 (6 bits = 64 values)
            for x in range(pow(2, 6)):
                
                row, column = calcRowColumnSbox(s, E_R15, x)
                rowF, columnF = calcRowColumnSbox(s, E_R15f, x)
                
                if (solutionIsValid(s, P_1_L16_xor_L16f, row, column, rowF, columnF)):
                    currSols.append(x)
            sol["sbox{}".format(s)].append(currSols)

        # the solution is the only block that is common to every set of solutions for the current sbox
        solution = common(sol["sbox{}".format(s)])
        K16 = K16 << 6
        K16 = K16 | solution

        if verbose is True:
            print("Sbox", s + 1)
            print("Potential solutions :")
            pp.pprint(sol['sbox{}'.format(s)])
            print("Solution", s+1 , "=", hex(solution))
            print("current K16 =", hex(K16))
            
    return K16

K16 = findK16(ciphertext, faultedCyphertexts, verbose=True)

Indexes of the SBoxes to be attacked:
{   'sbox0': [0, 27, 28, 29, 30, 31],
    'sbox1': [23, 24, 25, 26, 27, 28],
    'sbox2': [19, 20, 21, 22, 23, 24],
    'sbox3': [15, 16, 17, 18, 19, 20],
    'sbox4': [11, 12, 13, 14, 15, 16],
    'sbox5': [7, 8, 9, 10, 11, 12],
    'sbox6': [3, 4, 5, 6, 7, 8],
    'sbox7': [0, 1, 2, 3, 4, 31]}

Sbox 1
Potential solutions :
[   [2, 4, 5, 6, 14, 27, 34, 36, 37, 38, 46, 59],
    [4, 5, 20, 21, 36, 37],
    [37, 39, 44, 46],
    [10, 14, 16, 20, 33, 37, 41, 45, 59, 63],
    [3, 11, 33, 37, 41, 45, 53, 61],
    [7, 9, 12, 23, 25, 28, 37, 44, 45, 53, 60, 61]]
Solution 1 = 0x25
current K16 = 0x25
Sbox 2
Potential solutions :
[   [2, 3, 44, 45, 46, 47],
    [20, 21, 22, 23, 33, 35, 44, 45, 46, 47],
    [0, 4, 16, 20, 32, 33, 36, 37, 42, 46],
    [19, 27, 37, 38, 45, 46],
    [3, 13, 15, 19, 29, 31, 45, 46, 61, 62],
    [3, 14, 35, 46]]
Solution 2 = 0x2e
current K16 = 0x96e
Sbox 3
Potential solutions :
[   [16, 17, 18, 19, 32, 33, 34, 35, 42, 43, 48, 49, 

Retrieving K from K16

Now we have 48 bits that are at the right position, and 16 bits that are wrong. Among those 16 bits, 8 are parity bits that are useless for the DES computation. This leaves us with 8 bits to bruteforce (256 combinations).

In the next step we will try to encrypt with every possible combination and compare the output with the cyphertext we have. If the results are the same, we found the 8 matching bits ie we found the key.

In [38]:
from itertools import product

def setKthBit(n,k): 
    return ((1 << k) | n) 

PC1inv = [
    8,  16, 24, 56, 52, 44,
    36, 0,  7,  15, 23, 55,
    51, 43, 35, 0,  6,  14,
    22, 54, 50, 42, 34, 0,
    5,  13, 21, 53, 49, 41,
    33, 0,  4,  12, 20, 28,
    48, 40, 32, 0,  3,  11,
    19, 27, 47, 39, 31, 0,
    2,  10, 18, 26, 46, 38,
    30, 0,  1,  9,  17, 25,
    45, 37, 29, 0
]

# Bits 9, 18, 22, 25, 35, 38, 43 et 54 are lost and thus put to 0 here
PC2inv = [
    5,  24, 7,  16, 6,  10,
    20, 18, 0,  12, 3,  15,
    23, 1,  9,  19, 2,  0,
    14, 22, 11, 0,  13, 4,
    0,  17, 21, 8,  47, 31,
    27, 48, 35, 41, 0,  46,
    28, 0,  39, 32, 25, 44,
    0,  37, 34, 43, 29, 36,
    38, 45, 33, 26, 42, 0,
    30,  40
]

def findK56(plaintext, ciphertext, K16):
    # We put the 48 bits of K16 at their position in K (going backwards in the key schedule)
    # Inverse of PC2 : 48 -> 56 bits, 8 wrong bits which's positions are known
    # Inverse of PC1 : 56 -> 64 bits, 8 wrong bits (parity, no consequence on the calculation). However our 8 wrong 
    # bits from PC2 are still wrong, but in different positions that are still trackable
    K48 = permutation(permutation(K16, PC2inv, 48), PC1inv, 56)
        
    # Bruteforcing the 8 missing bits
    masques = list(product([0, 1], repeat=8))
    positionsToRecover = [50, 49, 45, 44, 13, 10, 6, 4]
    
    for mask in masques:
        hypothesis = 0
        for i in range(8):
            if mask[i] == 1:
                hypothesis = setKthBit(hypothesis, k=positionsToRecover[i])
        hypothesis |= K48
        if ciphertext == DES(plaintext, hypothesis):
            return hypothesis
    
    print("Impossible de retrouver K56, erreur")
    raise

K56 = findK56(plaintext, ciphertext, K16)

def parity(K56b):
    strCle = bin(K56b)[2:]
    while len(strCle) < 64:
        strCle = "0" + strCle
    cle = ''
    i = 0
    while i < 64:
        currByte = strCle[i:i+8]
        if currByte.count('1') % 2 == 0:
            currByte = currByte[:-1] + '1'
        cle += currByte
        i+=8
    return int(cle, 2)

print("Found Key:", hex(parity(K56)))
print("Verifying the correctness:")
print()
print("Computed cypher:", hex(DES(plaintext, parity(K56))))
print("Right cypher:   ", hex(ciphertext))

Found Key: 0x375be6f82cad155b
Verifying the correctness:

Computed cypher: 0xf2ea8be913629cbd
Right cypher:    0xf2ea8be913629cbd


For input plaintext = 0x54bd2a11e5938c37 and ciphertext = 0xf2ea8be913629cbd, we get key = 0x375be6f82cad155b.

We can verify the result by using online DES calculator http://www.emvlab.org/descalc/

The bruteforced key is not always identical to the original keySince the DES operation (both encryption and decryption) ignores the LSB of each byte of the key. That is, if we flip any of the LSBs within the key, the operation remains the same. For example, two different keys 0x375BE6F82CAD155B and 0x365AE7F92DAC145A produce output (ciphertext) = 0xF2EA8BE913629CBD from the input (plaintext) = 0x54BD2A11E5938C37

# Thoughts


DFA is a useful tool, but not very practical. 

DFA requires physical posession of a target device, but it is possible to inject a fault remotely. DFA also requires deep knowledge and understanding of the underlying system; performing this type of attack without knowing the structure and operation of the system and its behavior is near-impossible. 

It is important to secure your devices from an attaker. Without having physical access to your device, it would be very difficult for the attaker to perform DFA. However, if the attaker gains access, there are a lot of others different vulnerabilities, that he can exploit, such as hardware trojans, perform cold boot attacks, etc.